## Simple Hand Written Numbers Classifier

Another classification example to recognize hand written numbers. Trained using MNIST dataset.

After training the model, you can use Sample_Sketching sketch in Processing to write numbers and see the results here.

In [ ]:
#install a required library to use OSC protocol
!pip install https://github.com/attwad/python-osc/archive/master.zip
!pip install matplotlib

In [ ]:
#setup OSC Protocol to communicate with Processing
import OSCHelper
server=OSCHelper.createServer(9000)

In [ ]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model

def create_model(nbclasses,firstLayer,layers):
    model=models.Sequential()
    model.add(layers.Dense(firstLayer,activation='relu',input_shape=(784,)))
    for l in layers:
        model.add(layers.Dense(l,activation='relu'))
    model.add(layers.Dense(nbclasses,activation='softmax'))
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
#setup training data. Here we using MNIST dataset for numbers
# the training set is 28x28 input, and a number 0~9 as output

from tensorflow.keras.datasets import mnist
from tensorflow.keras import utils

#load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

#scale training input from 0~255 -> 0~1 by dividing on 255 
x_train=x_train/255.0
x_test=x_test/255.0

nb_classes=10

#convert the input from 2D arrays to 1D arrays
X_train = x_train.reshape(x_train.shape[0], 28*28)
X_test = x_test.reshape(x_test.shape[0], 28*28)

#convert numbers output to one-hot encoding
Y_train = utils.to_categorical(y_train, nb_classes)
Y_test = utils.to_categorical(y_test, nb_classes)

In [ ]:
print(X_train.shape)

In [ ]:
#plot an example image from the dataset
%matplotlib inline
import matplotlib.pyplot as plt

index=10
plt.imshow(x_train[index],cmap='gray')
print(y_train[index])

In [ ]:
TrainModel=True #Set to False to load pretrained model
if TrainModel:
    #Either train your own model
    model=create_model(nb_classes,64,[64,32,16])
    model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=128)
else:
    #or just load a pretrained model (trained for model arch (64,[64,32,16]) with 97% accuracy)
    model=create_model(nb_classes,64,[64,32,16]) #Don't change the layers
    model.load_weights("model.h5")

In [ ]:
import numpy as np

def onOSC_Image(*args):
    width=args[1]
    height=args[2]
    args=args[3:]
    img=np.reshape(args,(width,height))
    x=np.reshape(img,(width*height))
    res=model.predict(np.array([x]))
    plt.imshow(img,cmap='gray')
    plt.title("Prediction:{0}".format(np.argmax(res)))
    plt.show()

server.addMsgHandler( "/inputs/image", onOSC_Image )

OSCHelper.start_server(server)


In [ ]:
server.close()
